# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [219]:
from abc import ABC, abstractmethod
from collections.abc import Iterable, Mapping
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, \
    PegasusTokenizerFast, pipeline

from htools import *
from incendio.utils import DEVICE, gpu_setup

In [4]:
gpu_setup(False)

cpu


/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:38: UserWarning: Cuda not available.
  if not torch.cuda.is_available(): warnings.warn('Cuda not available.')
/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:41: UserWarning: Incendio device is not cuda.
  warnings.warn('Incendio device is not cuda.')


In [2]:
version = 'tuner007/pegasus_paraphrase'
net = PegasusForConditionalGeneration.from_pretrained(version)
tok = PegasusTokenizerFast.from_pretrained(version)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60954), raddr=('52.216.233.221', 443)>
  self._sock = None
/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60955), raddr=('52.216.233.221', 443)>
  self._sock = None


/Users/hmamin/anaconda3/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:69: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/hmamin/.cache/torch/transformers/fa4532c0035b101d7abcd5c0c9c34a83288902b66c5616034db1a47643e05c75.efce77b8dcd2c57b109b0d10170fcdcd53f23c21286974d4f66706536758ab6e'>
  m.ParseFromString(open(filename, "rb").read())


In [39]:
_ = net.to(DEVICE)

In [50]:
texts = [
    'Educational games and digital learning materials to provide K-12 '
    'students with enriching experiences.',
    'The world\'s largest social network. Helping people build and maintain '
    'relationships in a disconnected world.',
    'I hate school. I wish my teacher would leave me alone. I don\'t think '
    ' he likes me.',
    'Today the president announced new plans to revamp the private '
    'healthcare system. Pundits questioned how he would manage to pass the '
    'bill.'
]

In [5]:
hdir(tok)

Using bos_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.


{'SPECIAL_TOKENS_ATTRIBUTES': 'attribute',
 'add_special_tokens': 'method',
 'add_tokens': 'method',
 'additional_special_tokens': 'attribute',
 'additional_special_tokens_ids': 'attribute',
 'all_special_ids': 'attribute',
 'all_special_tokens': 'attribute',
 'all_special_tokens_extended': 'attribute',
 'backend_tokenizer': 'attribute',
 'batch_decode': 'method',
 'batch_encode_plus': 'method',
 'bos_token': 'attribute',
 'bos_token_id': 'attribute',
 'build_inputs_with_special_tokens': 'method',
 'clean_up_tokenization': 'method',
 'cls_token': 'attribute',
 'cls_token_id': 'attribute',
 'convert_ids_to_tokens': 'method',
 'convert_tokens_to_ids': 'method',
 'create_token_type_ids_from_sequences': 'method',
 'decode': 'method',
 'deprecation_warnings': 'attribute',
 'encode': 'method',
 'encode_plus': 'method',
 'eos_token': 'attribute',
 'eos_token_id': 'attribute',
 'from_pretrained': 'method',
 'get_added_vocab': 'method',
 'get_special_tokens_mask': 'method',
 'get_vocab': 'metho

In [6]:
res = tok.prepare_seq2seq_batch(texts, truncation=True, padding='longest')

In [7]:
res

{'input_ids': tensor([[11263,   727,   111,  1016,   761,   917,   112,   319,  1046,  6054,
           392,   122, 26838,  1747,   107,     1,     0,     0,     0,     0,
             0,     0,     0],
        [  139,   278,   131,   116,  1368,   525,   952,   107, 22844,   200,
           736,   111,  1634,  2074,   115,   114, 20402,   278,   107,     1,
             0,     0,     0],
        [  125,  4180,   399,   107,   125,  1216,   161,  2118,   192,   858,
           213,  1600,   107,   125,   272,   131,   144,   311,   178,  5606,
           213,   107,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
gen = net.generate(**res)

tensor([[   0, 1046, 6054,  392,  137,  207, 2387,  727,  111, 1016,  761,  917,
          107,    1,    0,    0,    0],
        [   0,  139,  278,  131,  116, 1368,  525,  952,  107,    1,    0,    0,
            0,    0,    0,    0,    0],
        [   0,  125,  272,  131,  144,  172,  399,  111,  161, 2118,  591,  131,
          144,  172,  213,  107,    1]])

In [19]:
tok.batch_decode(gen.tolist(), skip_special_tokens=True)

['K-12 students can use educational games and digital learning materials.',
 "The world's largest social network.",
 "I don't like school and my teacher doesn't like me."]

In [54]:
@add_docstring(net.generate)
def paraphrase(text, n=1, temperature=1.5, **gen_kwargs):
    batch = tok.prepare_seq2seq_batch([text], truncation=True, 
                                      padding='longest').to(DEVICE)
    paraphrased = net.generate(**batch, num_return_sequences=n,
                               temperature=temperature, **gen_kwargs)
    return tok.batch_decode(paraphrased.tolist(), skip_special_tokens=True)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
paraphrase(texts[3], n=5, num_beams=10)

['The president announced plans to reform the private healthcare system.',
 'Pundits wondered how the president would pass the bill.',
 'Pundits questioned how the president would pass the bill.',
 'Pundits were questioning how the president would pass the bill.',
 'Pundits questioned how he would get the bill passed.']

In [53]:
texts[-1]

'Today the president announced new plans to revamp the private healthcare system. Pundits questioned how he would manage to pass the bill.'

In [48]:
net.generate()

{'input_ids': <Parameter "input_ids: Union[torch.LongTensor, NoneType] = None">,
 'decoder_input_ids': <Parameter "decoder_input_ids: Union[torch.LongTensor, NoneType] = None">,
 'max_length': <Parameter "max_length: Union[int, NoneType] = None">,
 'min_length': <Parameter "min_length: Union[int, NoneType] = None">,
 'do_sample': <Parameter "do_sample: Union[bool, NoneType] = None">,
 'early_stopping': <Parameter "early_stopping: Union[bool, NoneType] = None">,
 'num_beams': <Parameter "num_beams: Union[int, NoneType] = None">,
 'temperature': <Parameter "temperature: Union[float, NoneType] = None">,
 'top_k': <Parameter "top_k: Union[int, NoneType] = None">,
 'top_p': <Parameter "top_p: Union[float, NoneType] = None">,
 'repetition_penalty': <Parameter "repetition_penalty: Union[float, NoneType] = None">,
 'bad_words_ids': <Parameter "bad_words_ids: Union[Iterable[int], NoneType] = None">,
 'bos_token_id': <Parameter "bos_token_id: Union[int, NoneType] = None">,
 'pad_token_id': <Para

In [58]:
class ParaphrasePipeline:
    
    name = 'tuner007/pegasus_paraphrase'
    
    def __init__(self, net=None, tok=None):
        self.net = net or PegasusForConditionalGeneration.from_pretrained(
            self.name).to(DEVICE)
        self.tok = tok or PegasusTokenizerFast.from_pretrained(self.name)
        
    @add_docstring(PegasusForConditionalGeneration.generate)
    def __call__(self, text, n=1, temperature=1.5, **kwargs):
        # TODO: not sure how many rows of text can be done in a single batch.
        # Maybe look at other pipelines to see what they do. I'm thinking we
        # could auto-batch longer sequences (e.g. a list w/ 10_000 strings
        # might become 100 batches of 100).
        texts = [text] if isinstance(text, str) else text
        batch = self.tok.prepare_seq2seq_batch(texts, truncation=True, 
                                               padding='longest').to(DEVICE)
        gen_tokens = self.net.generate(**batch, num_return_sequences=n,
                                       temperature=temperature, **kwargs)
        gen = self.tok.batch_decode(gen_tokens.tolist(), 
                                    skip_special_tokens=True)
        if not isinstance(text, str) and len(text) > 1: 
            gen = [gen[i*n:(i+1)*n] for i in range(len(text))]
        return gen

In [51]:
pipe = ParaphrasePipeline(net, tok)

In [52]:
pipe('It was a beautiful rainy day.')

['There was a beautiful day.']

In [53]:
pipe('It was a beautiful rainy day.', n=3)

['There was a beautiful day.',
 'It was a nice day.',
 'It was raining but it was nice.']

In [54]:
pipe(['It was a beautiful rainy day.', 'The duck was yellow and fluffy.'])

[['There was a beautiful day.'], ['The duck was fluffy and yellow.']]

In [55]:
pipe(['It was a beautiful rainy day.', 'The duck was yellow and fluffy.'], 
     n=3)

[['There was a beautiful day.',
  'It was a nice day.',
  'It was raining but it was nice.'],
 ['The duck was fluffy and yellow.',
  'The duck was fluffy.',
  'The duck was big and fluffy.']]

In [56]:
pipe(['It was a beautiful rainy day.'], n=2)

['There was a beautiful day.', 'It was a nice day.']

In [63]:
class TransformerTransform:
    
    def __init__(self, mode, n=1):
        self.mode = mode
        self.n = n
        self.predictor = self._get_transformer()
        
#     def __call__(self, text):
#         self._preprocess(text)
    
#     @abstractmethod
#     def _preprocess(self, text):
#         pass
    
    def _get_transformer(self):
        if self.mode == 'mask':
            return pipeline('fill-mask')
        elif self.mode == 'generate':
            return pipeline('text-generation')
        elif self.mode == 'paraphrase':
            return ParaphrasePipeline()
        else:
            raise ValueError('mode must be in (mask, generate, paraphrase).')

In [242]:
class FillMaskTransform:    
    
    def __init__(self, pipe=None):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or pipeline('fill-mask')
        assert type(self.pipe).__name__ == 'FillMaskPipeline'
    
    def _preprocess(self, text, n=1, min_keep=3):
        """min_keep overrides n if they conflict."""
        if listlike(text):
            return [self._preprocess(row, n, min_keep) for row in text]
        
        tokens = text.split()
        idx = np.random.choice(range(len(tokens)), 
                               min(len(tokens)-min_keep, n),
                               replace=False)
        return ' '.join(t if i not in idx else MASK 
                        for i, t in enumerate(tokens))
    
    def __call__(self, text, n=1, min_keep=3, return_all=False):
        # Each item will be a list of strings. Each string in res[i]
        # will have i words changed. If text is a sequence of strings, we must
        # handle each one separately because each is passed through pipeline
        # repeatedly.
        if listlike(text):
            return [self(row, n, min_keep, return_all) for row in text]
        
        res = [[text]]
        for i in range(n):
            text = [enc['sequence'].replace('<s>', '').replace('</s>', '')
                    for row in res[-1] for enc in 
                    self.pipe(self._preprocess(row, min_keep=min_keep))]
            res.append(text)
        if not return_all: res = res[n]
        return res

In [236]:
MASK = '<mask>'

In [237]:
t = 'I went to the store today to buy eggs.'
ts = [t, 'The bird swooped down onto the picnic table and squawked loudly.']

In [238]:
# m_pipe = pipeline('fill-mask')
masker = FillMaskTransform(m_pipe)

In [239]:
masker._preprocess(t)

'I went <mask> the store today to buy eggs.'

In [240]:
masker._preprocess(ts)

['I went to the store today to <mask> eggs.',
 'The bird swooped down onto <mask> picnic table and squawked loudly.']

In [241]:
masker._preprocess(ts, n=3)

['I <mask> to <mask> store today <mask> buy eggs.',
 'The bird swooped down onto <mask> picnic table and <mask> <mask>']

In [244]:
masker(t)

['I went to the store today to buy eggs.',
 'We went to the store today to buy eggs.',
 'She went to the store today to buy eggs.',
 'He went to the store today to buy eggs.',
 'Someone went to the store today to buy eggs.']

In [245]:
masker(t, 2)

['I went to the store today to buy eggs.',
 'I headed to the store today to buy eggs.',
 'I drove to the store today to buy eggs.',
 'I came to the store today to buy eggs.',
 'I head to the store today to buy eggs.',
 'I went to the store today to purchase...',
 'I went to the store today to purchase.',
 'I went to the store today to purchase:',
 'I went to the store today to purchase…',
 'I went to the store today to purchase....',
 'I went to the store today to get eggs.',
 'I went to the store today and get eggs.',
 'I went to the store today & get eggs.',
 'I went to the store today I get eggs.',
 'I went to the store today, get eggs.',
 'I went to the store today to collect eggs.',
 'I drove to the store today to collect eggs.',
 'I headed to the store today to collect eggs.',
 'I returned to the store today to collect eggs.',
 'I came to the store today to collect eggs.',
 'I went to the store today to buy eggs.',
 'I went to the store today to purchase eggs.',
 'I went to the s

In [246]:
masker(t, 1, return_all=True)

[['I went to the store today to buy eggs.'],
 ['I went to the store today to buy eggs.',
  'I went to the store today to purchase eggs.',
  'I went to the store today to get eggs.',
  'I went to the store today to collect eggs.',
  'I went to the store today to grab eggs.']]

In [247]:
masker(ts)

[['I went to the supermarket today to buy eggs.',
  'I went to the store today to buy eggs.',
  'I went to the grocery today to buy eggs.',
  'I went to the mall today to buy eggs.',
  'I went to the butcher today to buy eggs.'],
 ['The bird swooped down onto the picnic table and squawked loudly.',
  'The bird swooped up onto the picnic table and squawked loudly.',
  'The bird swooped overhead onto the picnic table and squawked loudly.',
  'The bird swooped gently onto the picnic table and squawked loudly.',
  'The bird swooped softly onto the picnic table and squawked loudly.']]

In [248]:
masker(ts, n=2)

[['I went to the store yesterday to buy eggs.',
  'I went to the store intending to buy eggs.',
  'I went to the store wanting to buy eggs.',
  'I went to the store today to buy eggs.',
  'I went to the store expecting to buy eggs.',
  'I went to the store today to purchase eggs.',
  'We went to the store today to purchase eggs.',
  'She went to the store today to purchase eggs.',
  'Someone went to the store today to purchase eggs.',
  'He went to the store today to purchase eggs.',
  'I went to the store yesterday to get eggs.',
  'I went to the store wanting to get eggs.',
  'I went to the store today to get eggs.',
  'I went to the store intending to get eggs.',
  'I went to the store expecting to get eggs.',
  'I went to grocery store today to collect eggs.',
  'I went to the store today to collect eggs.',
  'I went to my store today to collect eggs.',
  'I went to a store today to collect eggs.',
  'I went to another store today to collect eggs.',
  'I went to the supermarket tod

In [249]:
class ParaphraseTransform:
    """Not sure how useful this will really be but this basically just 
    wraps ParaphrasePipeline to share a more similar interface with the other
    NLP transforms.
    """

    def __init__(self, pipe=None):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or ParaphrasePipeline()
        assert type(self.pipe).__name__ == 'ParaphrasePipeline'
        
    def _preprocess(self, text):
        return text
    
    def __call__(self, text, n=1, **kwargs):
        return self.pipe(text, n=n, **kwargs)

In [73]:
p_tfm = ParaphraseTransform(pipe)

In [74]:
p_tfm('The beach is loud and crowded today.', n=3)

['The beach is crowded.',
 'There is a lot of people on the beach.',
 'There is a lot of people at the beach.']

In [174]:
p_tfm(ts)

[['I went to buy eggs.'], ['The bird swooped down onto the picnic table.']]

In [175]:
p_tfm(ts, n=2)

[['I went to buy eggs.', 'I went to the store to buy eggs.'],
 ['The bird swooped down onto the picnic table.',
  'The bird swooped down on the picnic table.']]

In [227]:
def listlike(x):
    """Checks if an object is a list/tuple/set/array etc. Strings and
    mappings (e.g. dicts) are not considered list-like.
    """
    return isinstance(x, Iterable) and not isinstance(x, (str, Mapping))

In [226]:
for obj in ('a', 6, [], (), {}, set(), [3, 4], ['a', 'b'], ('a',), {'a': 'b'},
            {'a', 'b'}, np.arange(5)):
    print(type(obj), listlike(obj))

<class 'str'> False
<class 'int'> False
<class 'list'> True
<class 'tuple'> True
<class 'dict'> False
<class 'set'> True
<class 'list'> True
<class 'list'> True
<class 'tuple'> True
<class 'dict'> False
<class 'set'> True
<class 'numpy.ndarray'> True


In [250]:
class GenerativeTransform:
    
    def __init__(self, pipe=None):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or pipeline('text-generation')
        assert type(self.pipe).__name__ == 'TextGenerationPipeline'
    
    def _preprocess(self, text, drop=None, drop_pct=None, rand_low=None, 
                    rand_high=None, min_keep=3, return_tuple=False):
        """Truncate text."""
        if listlike(text):
            return [self._preprocess(row, drop, drop_pct, rand_low, rand_high,
                                     min_keep, return_tuple) for row in text]
        
        tokens = text.split()
        if len(tokens) <= min_keep:
            n = 0
        else:
            # Default is to truncate the last 20% of the sequence.
            if drop:
                n = drop
            elif drop_pct:
                n = int(drop_pct * len(tokens))
            elif rand_low is not None and rand_high is not None:
                n = np.random.randint(rand_low, rand_high)
            else:
                n = int(np.ceil(.2 * len(tokens)))
            n = np.clip(n, 0, len(tokens) - min_keep)
            tokens = tokens[:-n]
        truncated = ' '.join(tokens)
        return (truncated, n) if return_tuple else truncated
    
    def __call__(self, text, n=1, min_length=3, max_length=8, 
                 **generate_kwargs):
        if listlike(text):
            return [self(row, n, min_length, max_length, **generate_kwargs) 
                    for row in text]
    
        # generate counts current length as part of min_length. 
        # For now, we're just splitting words so I'm naively inflating n_curr 
        # a bit since pipeline does some form of sub-word tokenization.
        text = self._preprocess(text)
        n_curr = int(len(text.split()) * 1.1)
        res = self.pipe(text, min_length=n_curr + min_length,
                        max_length=n_curr + max_length,
                        num_return_sequences=n, **generate_kwargs)
        return [row['generated_text'] for row in res]

In [251]:
# g_pipe = pipeline('text-generation')
g_tfm = GenerativeTransform(g_pipe)

In [254]:
g_tfm._preprocess(t)

'I went to the store today to'

In [255]:
g_tfm._preprocess(ts, return_tuple=True)

[('I went to the store today to', 2),
 ('The bird swooped down onto the picnic table', 3)]

In [256]:
g_tfm(t)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['I went to the store today to buy my groceries. My son went and']

In [257]:
g_tfm(t, n=2)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['I went to the store today to ask to meet with the clerk I think',
 'I went to the store today to sell them. It seemed like a well']

In [258]:
g_tfm(ts)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[['I went to the store today to pick up a couple of pieces, and'],
 ['The bird swooped down onto the picnic table on the third floor and took some']]

In [259]:
g_tfm(ts, n=2)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[['I went to the store today to pick up a pair of shoes with the',
  'I went to the store today to get up because I heard there was some'],
 ['The bird swooped down onto the picnic table across from me before he could get',
  'The bird swooped down onto the picnic table and made a short descent in a']]